<a href="https://colab.research.google.com/github/swlee123/Food-Vision-Computer-Vision/blob/main/Food_Vision_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 1 :Food Vision Big

See the template of this notebook on Github : https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/TEMPLATE_07_food_vision_milestone_project_1.ipynb

In [ ]:
# Downgrade Tensorflow Version (run this in Google Colab) 
# !pip install tensorflow==2.4.1
import tensorflow as tf
tf.__version__

AttributeError: ignored

## Check GPU

Google Colab offers free GPUs ,however , not all of them are compatible with [mixed precision](https://www.tensorflow.org/guide/mixed_precision) training.

Google Colab offers:
* k80 (not compatible)
* P100 (not compatible)
* Tesla T4 (compatible)

Knowing this, in order to use mixed precision training we need access to a Tesla T4 (from Colab) of if we're using our own hardware, our GPU needs a score of 7.0+  https://developer.nvidia.com/cuda-gpus#compute

In [ ]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Get helper functions

In past modules, we've created a bunch of helper functions to do small tasks required for our notebooks.
Rather than rewrite all of there, we can import a script and load them in from there.
The script we've got available can be found on Github https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
# Dowload helper function scripts

! wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-12-27 08:38:21--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.3’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-12-27 08:38:21 (67.9 MB/s) - ‘helper_functions.py.3’ saved [10246/10246]



In [ ]:
# import series of helper functions for the notebook

from helper_functions import create_tensorboard_callback,plot_loss_curves,compare_historys

AttributeError: ignored

## Use Tensorflow Dataset to download data

If you want to get an overview on tensorflow dataset read https://www.tensorflow.org/datasets/overview

In [ ]:
# Get tensorflow dataset
import tensorflow_datasets as tfds

In [ ]:
# list all available datasets
dataset_list = tfds.list_builders() # get all available datasets in TFDS
print("food101" in dataset_list) # is our target dataset in the dataset_list?

In [ ]:
# Load in the data (takes 5-6 mins in Google Colab)
(train_data,test_data),ds_info = tfds.load(name="food101",
                                   split=["train","validation"],
                                   shuffle_files =True,
                                   as_supervised = True, # data gets returned in tuple format (data,label)
                                   with_info = True)

## Exploring our dataset

We need to find:
* Class names
* The shape of our input data (image tensor)
* The datatype of our input data
* What the labels looks like (one-hot encoded or label encoded)
* Do the labels match up with the class names?

In [ ]:
# Features of Food101 from TFDS
ds_info.features

In [ ]:
# Get the class names 
class_names = ds_info.features["label"].names

In [ ]:
class_names[:10]

In [ ]:
# Take one sample of train data
train_one_sample = train_data.take(1) # (image_tensor,label)

In [ ]:
# What does one sample of our training data look like?
train_one_sample

In [ ]:
# output info of our training sample

for image,label in train_one_sample:
  print(f"""
  Image shape : {image.shape},
  Image datatype : {image.dtype},
  Target class from Food101 (tensor form) : {label}
  Class name (str form) : {class_names[label.numpy()]}

  """)

In [ ]:
# what image tensor look like ?
image

In [ ]:
import tensorflow as tf

# max and min in image tensor (usually 0-255)
tf.reduce_min(image),tf.reduce_max(image)

### Plot an image from tensorflow dataset



In [ ]:
# plot an image tensor

import matplotlib.pyplot as plt

plt.figure()
plt.imshow(tf.cast(image,dtype=tf.float32)/255.)
plt.title(f"{class_names[label.numpy()]}")
plt.axis(False)
plt.show()


In [ ]:
# make a function to random a image from the dataset 

def random_image():
  # Take one sample of train data
  train_one_sample = train_data.take(1) # (image_tensor,label)
  # output info of our training sample

  for image,label in train_one_sample:
    print(f"""
    Image shape : {image.shape},
    Image datatype : {image.dtype},
    Target class from Food101 (tensor form) : {label}
    Class name (str form) : {class_names[label.numpy()]}
    """)
  # plot an image tensor

  import matplotlib.pyplot as plt

  plt.figure()
  plt.imshow(tf.cast(image,dtype=tf.float32)/255.)
  plt.title(f"{class_names[label.numpy()]}")
  plt.axis(False)
  plt.show()




In [ ]:
random_image() # you can rerun this cell multiple times to look at our data 

## Create preprocessing functions for our data

Neural Network perform best when data is in a certain way(e.g. batched, normalized, etc)

However, not all data (including data from Tensorflow Datasets) comes like this.

So in order to get it ready for a neural netowrk, you'll often have to write preprocessing functions and map it to your data.

What we know about our data:

* In `uint8` datatype
* Comprised of all different size tensors (different sized images)
* Not scaled( the pixel values are between 0 and 255)

What we know models like:

* Data in `float32` dtype (or for mixed precision `float16` and `float32`)
* For batches, Tensorflow likes all of the tensors within a batch to be of the same size
* Scaled (values between 0 and 1) also called normalized tensors generally perform better

With these points in mind, we've got a few things we can tacle with a preprocessing function

Since we're going to be using EfficientNetBX pretrained model from tf.keras.applications we dont need to rescale our data (these architectures have rescaling built-in)

This means our functions needs to:

 1. Reshape our images to same size
 2. Convert the dtype of image tensors from `unit8` to `floaat32`

In [ ]:
# Make a function for preprocessing images 
def preprocess_img(image,label,img_shape=224):
  """
  Converts image datatype from `uint8` -> `float32` and reshapes image to [img_shape,img_shape,color channels]
  """

  image = tf.image.resize(image,[img_shape,img_shape])
  # not required to scale it as EfficeintNet has a rescaling layer to do it
  image = tf.cast(image,dtype=tf.float32)
  return image,label # return tuple of (float32_img,label)


In [ ]:
# preprocess a single sample image and check it's output

preprocessed_img = preprocess_img(image,label)[0]

print(f"Before preprocessing : \n {image[:2]}...,\n Shape :{image.shape},\n Datatype :{image.dtype}")
print(f"After preprocessing : \n {preprocessed_img[:2]}...,\n Shape :{preprocessed_img.shape},\n Datatype :{preprocessed_img.dtype}")

## Batch and prepare dataset

Previously we use tf.image_from_directory().
Now we're going to make our data input pipeline going throught very fast , we're using examples from the guide https://www.tensorflow.org/guide/data_performance




In [ ]:
train_data

In [ ]:
# map preprocessing function to training (and parallelize)

train_data = train_data.map(map_func = preprocess_img,num_parallel_calls=tf.data.AUTOTUNE  )# utillize the cpu/gpu power

# shuffle train_data and turn it into batches and prefetch it (loaad faster)

train_data = train_data.shuffle(buffer_size = 1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)
# .shuffle buffer size shuffle 1000 imgs 1 time
# tf.data.AUTOTUNE is to find all computing power as possible to utillize it 


# map preprocessing functions to test data
test_data = test_data.map(map_func = preprocess_img,num_parallel_calls=tf.data.AUTOTUNE ).batch(32).prefetch(tf.data.AUTOTUNE)

# all of this method are to use to speed up our pipeline in our model

In [ ]:
train_data, test_data

## Create modelling callbacks

We're going to create a couple of callbacks to help us while our models trains:

* Tensorboard callback to log training results
* ModelCheckPoint callback to save our model's progress after feature extraction


In [ ]:
# create tensorboard callback (helper function .py)
from helper_functions import create_tensorboard_callback

# create model checkpoint callbacks to save our model's a progress during training
checkpoint_path = "model_checkpoint/ck.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_loss",
                                                      save_best_only=True,
                                                      save_weight_only=True,
                                                      verbose=0) 


In [ ]:
tf.__version__ # maybe will have buy occur for cells afterward

## Setup mixed precision training

For deeper understanding for mixed precision training
https://www.tensorflow.org/guide/mixed_precision

Mixed precision utillize the combination of float32 and floaat16 data types to speed up model performance

In [ ]:
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16") # set global data policy to mixed precision


In [ ]:
mixed_precision.global_policy() # check policy we are using

## build feature extraction model


You might found some bugs when running code below using tensorflow 2.5++ ,
the solution found is to downgraded the tensorflow version to <2.5 by running code below (run if bug is encountered)

In [ ]:
# Downgrade Tensorflow Version (run this in Google Colab) 
# !pip install tensorflow==2.4.1

In [ ]:
import tensorflow as tf
tf.__version__


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


# create base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# create functional model

input = layers.Input(shape=input_shape,name="input_layer")
# efficient net got built-in rescaling layer (optional can run code below)
# x = preprocessing.Rescale(1/255.) (x)

x = base_model(input,training=False) # put it in inference (not training ) mode

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(len(class_names))(x)

output = layers.Activation("softmax",dtype=tf.float32,name="softmax_activation")(x)

model = tf.keras.Model(input,output)


# compile the model

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(), # one-hot label - categorical
                                                            # more than one label = sparse
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model.summary()

## Checking layer dtype policies (are we using mixed precision?)

I am turning off mixed percision training for the sake of training time, it i use mixed precision, it cost like 100++ hours to train , but i surfed the discord someone came out with a solution to close the mixed precision

In [ ]:
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("float32") # set global data policy back to float32 policy


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


# create base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# create functional model

input = layers.Input(shape=input_shape,name="input_layer")
# efficient net got built-in rescaling layer (optional can run code below)
# x = preprocessing.Rescale(1/255.) (x)

x = base_model(input,training=False) # put it in inference (not training ) mode

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(len(class_names))(x)

output = layers.Activation("softmax",name="softmax_activation")(x)

model = tf.keras.Model(input,output)


# compile the model

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(), # one-hot label - categorical
                                                            # more than one label = sparse
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
# check if model is using mixed precision training layer by layer

for layer in model.layers:
  print(f"dtype policy: {model.dtype_policy}")


In [ ]:
# check info of layers in efficientnetb0 

for layer in model.layers[1].layers:
  print(layer.name,layer.trainable,layer.dtype_policy,layer.dtype)

print(f"Number of layers {len(base_model.layers)}")

In [ ]:
# check info of layers in efficientnetb0 

for layer in model.layers[1].layers:
  print(layer.name,layer.trainable,layer.dtype_policy,layer.dtype)

print(f"Number of layers {len(base_model.layers)}")

In [ ]:
# check if model is using mixed precision training layer by layer

for layer in model.layers:
  print(f"dtype policy: {model.dtype_policy}")


## Fit the feature extraction model

If our goal it to fine-tune a pretrained model, the general order of doing things is to:

1. Build a feature extraction model (train a couple output layer with base layers frozen
2. Fine-tune some of the frozen layers

## Real challenge starts here ...

In [ ]:
# another solution from comment section in udemy

!pip install tensorflow==2.11.0

In [ ]:
# fit the model

history_1_feature_extraction = model.fit(train_data,
                              epochs=3,
                              steps_per_epoch=len(train_data),
                              validation_data=test_data,
                              validation_steps = int(len(test_data)*0.10),
                              callbacks=[create_tensorboard_callback("training_log","efficient_net_ex1"),model_checkpoint])

In [ ]:
# evaluate on whole test data set

result_feature_extract = model.evaluate(test_data)
result_feature_extract

In [ ]:
# load model previously saved

model_2 = create_model()

model_2.load_weights("model_checkpoint/ck.ckpt")



In [ ]:
# check model performance 
import numpy as np

result_2 = model_2.evaluate(test_data)
np.isclose(result_2,result_feature_extract)

